In [9]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [10]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1,H2,H3, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2) 
        self.linear3 = nn.Linear(H2, H3) 
        self.linear4 = nn.Linear(H3, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        x = self.linear4(x)
        return x



In [12]:
import copy
import numpy as np
seed_torch(seed)
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, H3, D_out = 50, 5, 20, 15, 9, 3
model = Net(D_in, H1, H2, H3, D_out)
loss_fn = torch.nn.MSELoss(reduction='sum')
init_state = copy.deepcopy(model.state_dict())



In [13]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm

import xlrd
import pandas as pd
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)
book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_replicated = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

scaler = StandardScaler()
scaler.fit(data_unreplicated)
standardised_data = scaler.transform(data_replicated)

x_train = standardised_data[:, 0:5]
y_train = standardised_data[:, 5:8]

n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))



In [14]:
i = n_splits-1
test_batchsizes = [10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000]
test_lrs = [0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
model = Net(D_in, H1, H2, H3, D_out)
init_state = copy.deepcopy(model.state_dict())
for batchsize in test_batchsizes:
    for test_lr in test_lrs:
            N, D_in, H1, H2,H3, D_out = batchsize, 5, 20 ,15 ,9 , 3
            learning_rate=test_lr
            av_opt_epoch=0
            av_opt_loss=0
            for i, (train_idx, valid_idx) in enumerate(splits):
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                model.load_state_dict(init_state)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batchsize,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batchsize,
                                                           shuffle=False)
                UP=0
                epoch=0
                k=0
                opt_loss=1000
                old_avg_val_loss=0
                while UP<4 and epoch<300:
                    epoch+=1
                    k+=1
                    model.train()
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    model.eval()
                    avg_val_loss = 0.
                    for x_batch, y_batch in valid_loader:
                        y_pred = model(x_batch).detach()
                        avg_val_loss += (loss_fn(y_pred, y_batch).item() / 
                        (len(valid_loader.dataset)))
                    if k > 4:
                        if avg_val_loss> old_avg_val_loss:
                            UP+=1
                        if avg_val_loss<old_avg_val_loss:
                            UP=0
                        old_avg_val_loss=avg_val_loss
                        k=0
                    if avg_val_loss<opt_loss:
                        opt_loss = avg_val_loss
                        opt_epoch = epoch
                
                av_opt_epoch+=opt_epoch/n_splits
                av_opt_loss+=opt_loss/n_splits
            print(batchsize,test_lr,av_opt_epoch,av_opt_loss)


10 0.0001 29.666666666666668 2.0719595183266537
10 0.0002 14.166666666666668 2.0668419076336755
10 0.0003 41.666666666666664 1.8872614420784843
10 0.0004 31.5 1.7162609992424644
10 0.0005 19.833333333333332 1.8844458142254084
10 0.0006 51.5 2.0063736837440067
10 0.0007 7.333333333333333 2.082014009555181
10 0.0008 53.5 1.9047508890098996
10 0.0009 40.333333333333336 1.8406389227840634
10 0.001 6.333333333333334 2.010044865873125
10 0.002 26.5 1.6908967851930197
10 0.003 25.5 1.8991099261575277
10 0.004 17.5 1.9619858923885558
10 0.005 24.833333333333336 1.6501416342125999
10 0.006 25.166666666666664 1.873534408873982
10 0.007 18.666666666666664 2.184431071413888
10 0.008 5.5 2.2518505492475303
10 0.009 37.833333333333336 2.2807803796397312
10 0.01 54.33333333333333 2.36116404387686
20 0.0001 45.33333333333333 2.0794489932060243
20 0.0002 21.999999999999996 2.073979287412431
20 0.0003 49.00000000000001 1.7596579061614144
20 0.0004 10.833333333333332 2.0672160175111554
20 0.0005 42.0 1.7

100 0.008 18.666666666666664 2.0400714047749835
100 0.009 3.833333333333333 2.020091220008003
100 0.01 4.5 1.9121673976050482
200 0.0001 67.16666666666667 2.946653162638346
200 0.0002 69.5 2.4220233472188313
200 0.0003 47.166666666666664 2.4296746995713976
200 0.0004 52.83333333333333 2.185837624867757
200 0.0005 42.16666666666667 2.1896661949157714
200 0.0006 35.666666666666664 2.191034785376655
200 0.0007 31.666666666666664 2.197102703518338
200 0.0008 27.0 2.1946488783094615
200 0.0009 23.666666666666668 2.1962683868408197
200 0.001 21.833333333333336 2.1987237612406414
200 0.002 31.166666666666664 1.7970598623487686
200 0.003 19.666666666666664 1.8105630175272625
200 0.004 15.5 1.8036084005567763
200 0.005 10.666666666666666 1.8636262554592558
200 0.006 16.833333333333336 1.7964844502343074
200 0.007 6.999999999999999 1.9673876423305936
200 0.008 7.666666666666665 1.9647176170349123
200 0.009 9.833333333333334 1.964499782986111
200 0.01 8.333333333333334 1.7861092546251085
300 0.00

In [7]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN3.pt"))                
                                